# Seminar Notebook 2.3: Mixture Models

**LSE MY459: Computational Text Analysis and Large Language Models** (WT 2026)

**Ryan Hübert**

This notebook covers mixture models.

## The DGP of a simple mixture model

In lecture, we looked at a probabilistic model of a DGP in which documents are assigned to clusters; clusters differ in their word usage; and documents' token counts are drawn from a multinomial distribution. Let's create a simulated data set of a DFM that would be generated by such a model. We will use tools available in `numpy` to do the sampling and the matrix algebra. We'll also set the seed by creating a random number generator called `rng` with a seed of `2026`. Warning: if you run the following cells out of order (or re-run them more than once), you will no longer get the same numbers. Keep this in mind when you compare your results with what is written in the text.

We will consider a simple example of a corpus with a vocabulary that has 5 distinct types (i.e., pre-processed unigrams), which is organised into a latent structure of 3 clusters.

Given this structure, each document $i$ will be assigned to one of the three clusters, and each cluster $k$ will have a parameter $\boldsymbol{\mu}_k$ which will have five elements in it---one for each of the features. Recall that in the model from lecture, both $\boldsymbol{\pi}_i$ and $\boldsymbol{\mu}_k$ are drawn from distributions.

### Cluster feature probabilities 

Let's first examine $\boldsymbol{\mu}_k$, which under the model from class is drawn from a Dirichlet distribution with a parameter $\boldsymbol{\eta}$ that is a $J$ length vector. Let's draw a $\boldsymbol{\mu}_k$, using $\boldsymbol{\eta} = (1,1,1,1,1)$.

This randomly sampled $\boldsymbol{\mu}_k$ tells you how words are used in cluster $k$. Given that token counts will be sampled from a multinomial (details below), you can interpret this vector as saying: "When a token is drawn for a document, there is a 0.042 probability that token will be "cat", a 0.361 probability that token will be "dog", and so on."

Of course, we need to draw a $\boldsymbol{\mu}_k$ for every cluster $k \in \{1,2,...,K\}$. In this case, we need to draw $\boldsymbol{\mu}_1$, $\boldsymbol{\mu}_2$ and $\boldsymbol{\mu}_3$ since there are three clusters. We can do this by sampling as follows:

The `mu` object is a $K \times J$ matrix, where each row is one of the cluster's $\boldsymbol{\mu}_k$ vector. We can also do a little formatting to make it easier to see that these columns correspond to types in the vocabulary:


Looking at this matrix, you can already start to see some patterns that will emerge when we generate documents from this model. For example, documents in cluster 2 will be about twice as likely to have the word "fox" as documents in cluster 0 (Note: we are using Python indexing, which starts at zero!). That's because the probability that any given token will be "fox" is 0.33 in cluster 2 and 0.16 in cluster 0. You can also see that the top most used feature in clusters 0 and 1 is "dog", the top most used feature in cluster 2 is "fox".

These $\boldsymbol{\mu}_k$ vectors are drawn from a Dirichlet distribution, where its parameter $\boldsymbol{\eta}$ controls how "concentrated" word use is across the clusters. Let's see this by looking at the results from using different values for the prior. First, we'll again use a symmetric vector, but we'll have all values be much smaller than 1:

You can see that word use is now much more concentrated within a cluster. For example, in both clusters 0 and 1, "dog" is the most probable word to be used, and by a lot more than the previous example. Next let's use a symmetric vector, but we'll have all values be much greater than 1:

You can now see that word use is now much less concentrated within a cluster. In all the clusters, the probabilities of each word are much more similar, so document word use is more "random" than before. Finally, let's see what happens when we use an asymmetric $\boldsymbol{\eta}$, such as $\boldsymbol{\eta} = (10,1,1,1,1)$.

Now, you can see that in all three clusters, "cat" gets a lot of weight, reflecting the asymmetric prior. However, due to randomness (after all, these are still random draws), you still see the first document has a top feature that is not "cat" despite the lopsided $\boldsymbol{\eta}$. 

Of course, in very large samples (i.e., large numbers of clusters), you will see the average probability of the "cat" feature will dominate, as you can see below:

### Document cluster assignment

Under this model, $\boldsymbol{\pi}_i$ is drawn from a multinomial distribution with two parameters: 1 and $\boldsymbol{\alpha}$, where $\boldsymbol{\alpha}$, is a $K$-length vector giving the probability that a document will be assigned to each of the clusters. For example, let's consider a scenario where $\boldsymbol{\alpha} = (0.20, 0.40, 0.40)$, so that each document has a 0.2 probability of being assigned to cluster 0, a 0.4 probability of being assigned to cluster 1, and a 0.4 probability of being assigned to cluster 2.

For a hypothetical document $i$, let's draw that document's cluster.

Note here that the document is assigned to cluster 2. Since $\boldsymbol{\alpha}$ controls the probability that documents will be assigned to clusters, we can again see that in a very large sample, roughly 20% of documents will be in cluster 0, 40% in clusters 1 and 2, respectively:

### Generating a document

Now that we have each cluster's $\boldsymbol{\mu}_k$, _and_ we know which cluster our hypothetical document $i$ belongs to, we can randomly sample a vector of word counts for the hypothetical document $i$. According to the model, we use a multinomial distribution with two parameters: $M_i$ (document length) and $\boldsymbol{\pi}_i\boldsymbol{\mu}$. Due to matrix algebra (watch the indexing!) and since we saw the document was assigned to cluster 1, $\boldsymbol{\pi}_i\boldsymbol{\mu} = \boldsymbol{\mu}_1$, as you can see:

Finally, we can generate token counts for this hypothetical document, assuming the document has 10 tokens in it

### Generating a full DFM

Above, we simulated a single document. But can we simulate a full DFM? Yes! First, we need to define how many documents we want in our simulated DFM:

Now that we know how big we want our DFM, we can draw $\boldsymbol{\pi}_i$ for each of the documents, and then draw token counts for them.

## Run $k$-means on simulated data

We now have a simulated DFM where token usage in each document depends on cluster assignment (`pi`), and the clusters' word usage probabilities (`mu`). Now, let's pretend that this is a real dataset, and try to run $k$-means clustering on it to see if we can recover the cluster assignments we simulated above. Recall that each document's cluster assignment is given by:

We can also make this a bit easier for us to see by creating a `DataFrame` object where each row gives the known cluster (created from `pi`):

Now lets run $k$-means clustering, using the process from the previous notebook.

Now, let's add our _estimated_ cluster to the `cluster_assignment` object. 

Before proceeding note: $k$-means will assign _arbitrary_ labels to each cluster, so we need to match up the labels from our known clusters to our estimated clusters. The following code will do this using the [Hungarian algorithm](https://en.wikipedia.org/wiki/Hungarian_algorithm), although the details are not important. What is important to know is that k-means won't necessarily call the same clusters the same thing as you had called them when you created them.

Now that we have relabelled, let's see how many documents $k$-means accurately assigned to a cluster (relative to the actual cluster assignment from our simulated data).

$k$-means correctly recovered 58% of the clusters we assigned above. That's okay, but not _great_. Will we improve things by weighting and normalising our simulated DFM? 

Yes, this does better! But you may be able to do even better with other clustering methods. 